# Experiment 3: Variance

# TODO: 
1. Code this
2. Make it so we can optionally use data augmentation in the functions at the bottom of utils.py. I added it to the signature, but this is not yet implemented. 
3. Other than the stuff in BAL, this should be good for now

In [1]:
#Stuff for google colab

In [2]:
import numpy as np
import torch as th
import pandas as pd

import graphlearning as gl

import batch_active_learning as bal
import utils

#TODO: CHECK THE PARAMETERS IN bal.coreset_run_experiment

## Parameters and Function Definitions

In [3]:
#Non-Default Parameters

num_experiments = 2
num_epochs=1

save_path = "Experiment Results/Experiment 3/"

In [4]:
def experiment3(dataset, embedding, data_augmentation, num_experiments=num_experiments):
    assert dataset in utils.AVAILABLE_SAR_DATASETS, "Invalid dataset"
    assert dataset != "mstar", "Invalid dataset: not testing MSTAR"
    assert embedding in utils.AVAILABLE_EMBEDDINGS, "Invalid embedding"

    if embedding == "fine_tuned_tl":
        max_new_samples = bal.FINE_TUNED_MAX_NEW_SAMPLES_DICT[dataset]
    else:
        max_new_samples = bal.MAX_NEW_SAMPLES_DICT[dataset]

    acc_results = np.zeros(num_experiments)

    for i in range(num_experiments):
        # Do inside each experiment because we want to understand the impact of this kind of noise
        # Perform embedding
        if embedding == "cnnvae":
            X, labels, knn_data = utils.CNNVAE(dataset)
        elif embedding == "zero_shot_tl":
            X, labels, knn_data = utils.zero_shot_TL(
                dataset, data_augmentation=data_augmentation
            )
        else:
            X, labels, knn_data = utils.fine_tuned_TL(
                dataset, data_augmentation=data_augmentation, num_epochs=num_epochs
            )

        if isinstance(X, th.Tensor):
            X = X.numpy()
        if isinstance(labels, th.Tensor):
            labels = labels.numpy()

        # Create graph objects
        try:
            W = gl.weightmatrix.knn(
                X, utils.KNN_NUM, kernel="gaussian", knn_data=knn_data
            )
        except FileNotFoundError:
            return knn_data
        G = gl.graph(W)

        num_iter = max_new_samples // bal.BATCH_SIZE

        # Ensure each label is represented in core set
        initial = gl.trainsets.generate(labels, rate=1).tolist()

        coreset = bal.coreset_dijkstras(
            G,
            rad=bal.DENSITY_RADIUS,
            data=X,
            initial=initial,
            density_info=(True, bal.DENSITY_RADIUS, 1),
            knn_data=knn_data,
        )

        _, num_labels, acc_vals, _ = bal.coreset_run_experiment(
            X,
            labels,
            W,
            coreset,
            num_iter=num_iter,
            method="Laplace",
            display=False,
            use_prior=False,
            al_mtd="local_max",
            acq_fun="uc",
            knn_data=knn_data,
            mtd_para=None,
            savefig=False,
            batchsize=bal.BATCH_SIZE,
            dist_metric="angular",
            knn_size=utils.KNN_NUM,
            q=1,
            thresholding=0,
        )

        acc_results[i] = acc_vals[-1]
    end_labels = num_labels[-1]

    return {
        "num_points": end_labels,
        "experiments": num_experiments,
        "mean": np.mean(acc_results),
        "std_dev": np.std(acc_results),
    }


## Experiments
This is running 8 experiments each num_experiments times. This comes out to 400 runs for us. 

In [ ]:
all_results = {}

for data_augmentation in [True, False]:
    for dataset in utils.AVAILABLE_SAR_DATASETS[1:]:
        for embedding in utils.AVAILABLE_EMBEDDINGS[1:]:
            key = str(data_augmentation) + '_' + dataset + '_' + embedding
            print(key)
            experiment_result = experiment3(dataset, embedding, data_augmentation)
            all_results[key] = experiment_result.copy()
            if not isinstance(experiment_result, dict):
                assert False, "Error in the knn_data. Look at experiment_result. It contains knn_data now. "
            print(experiment_result)
            print()

True_open_sar_ship_zero_shot_tl
torch.Size([2296, 1, 128, 128])


Using cache found in /Users/jameschapman/.cache/torch/hub/pytorch_vision_v0.10.0
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([2296, 1, 128, 128])


Using cache found in /Users/jameschapman/.cache/torch/hub/pytorch_vision_v0.10.0


{'num_points': 748, 'experiments': 2, 'mean': 76.43847002090226, 'std_dev': 1.597705689692063}

True_open_sar_ship_fine_tuned_tl
Epoch 1/1
----------


...

Using cache found in /Users/jameschapman/.cache/torch/hub/pytorch_vision_v0.10.0
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X0_5_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X0_5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([4856, 1, 128, 128])


Using cache found in /Users/jameschapman/.cache/torch/hub/pytorch_vision_v0.10.0


{'num_points': 3121, 'experiments': 2, 'mean': 84.94637031199389, 'std_dev': 0.06803890990813244}

True_fusar_fine_tuned_tl
Epoch 1/1
----------
train Loss: 1.5596 Acc: 0.4239
val Loss: 1.5259 Acc: 0.4482

Training complete in 0m 56s
Best val Acc: 0.448181
torch.Size([4856, 1, 128, 128])
Epoch 1/1
----------
train Loss: 1.5797 Acc: 0.3695
val Loss: 1.5371 Acc: 0.4571

Training complete in 0m 54s
Best val Acc: 0.457104
torch.Size([4856, 1, 128, 128])


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/graphlearning/weightmatrix.py:90: RuntimeWarning: invalid value encountered in divide
  weights = np.exp(-4*D/eps[:,None])


In [ ]:
experiment_result

In [ ]:
new_save_path = save_path + 'all_results_' + str(num_experiments) 
if num_epochs < 10:
    new_save_path += '_testing'
df = pd.DataFrame.from_dict(all_results, orient='index')
df.to_pickle(new_save_path + '.pkl')

## Testing

In [4]:
import utils
import numpy as np
import torch as th

In [5]:
def tester(num_tests=10):
    out = []
    for i in range(10):
        print(f"Iteration {i+1}")
        X, labels, knn_data = utils.fine_tuned_TL('fusar', data_augmentation=False, num_epochs=1)
        num_zeros = len(np.where(knn_data[1][:, -1] == 0)[0])
        print(num_zeros)
        out.append(num_zeros)
    return out

In [6]:
out = tester()

Iteration 1
Epoch 1/1
----------
train Loss: 1.6151 Acc: 0.1030
val Loss: 1.6166 Acc: 0.0528

Training complete in 0m 15s
Best val Acc: 0.052848
4801
Iteration 2
Epoch 1/1
----------
train Loss: 1.6119 Acc: 0.1786
val Loss: 1.6079 Acc: 0.2107

Training complete in 0m 11s
Best val Acc: 0.210707
4825
Iteration 3
Epoch 1/1
----------
train Loss: 1.6110 Acc: 0.2142
val Loss: 1.6119 Acc: 0.1915

Training complete in 0m 11s
Best val Acc: 0.191489
4740
Iteration 4
Epoch 1/1
----------
train Loss: 1.6080 Acc: 0.2012
val Loss: 1.6055 Acc: 0.1682

Training complete in 0m 11s
Best val Acc: 0.168154
4714
Iteration 5
Epoch 1/1
----------
train Loss: 1.6136 Acc: 0.3054
val Loss: 1.6101 Acc: 0.3054

Training complete in 0m 11s
Best val Acc: 0.305422
4788
Iteration 6
Epoch 1/1
----------
train Loss: 1.6039 Acc: 0.3313
val Loss: 1.6054 Acc: 0.2841

Training complete in 0m 11s
Best val Acc: 0.284146
4720
Iteration 7
Epoch 1/1
----------
train Loss: 1.6061 Acc: 0.2145
val Loss: 1.6114 Acc: 0.0995

Traini

KeyboardInterrupt: 

In [ ]:
print(out)
#[0, 0, 0, 0, 0, 4712, 4677, 4669, 4680, 4672] with mps
#[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] with cpu

In [2]:
X, labels, knn_data = utils.fine_tuned_TL('fusar', data_augmentation=False, num_epochs=1)
print(X)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X0_5_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X0_5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/1
----------


/Users/jameschapman/Documents/GitHub/SAR_BAL/utils.py:349: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs = nn.functional.log_softmax(outputs)


train Loss: 1.6270 Acc: 0.0559
val Loss: 1.6237 Acc: 0.0494

Training complete in 0m 10s
Best val Acc: 0.049417
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [3]:
len(np.where(knn_data[1][:, -1] == 0)[0])

4650

In [4]:
X

array([[0.00590989, 0.        , 0.10457867, ..., 0.01400539, 0.        ,
        0.        ],
       [0.        , 0.00333806, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04948823, 0.01027269, 0.05872394, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.09199033, 0.        , ..., 0.00718634, 0.        ,
        0.        ],
       [0.02563937, 0.00534612, 0.11887628, ..., 0.03139264, 0.        ,
        0.        ],
       [0.00980122, 0.05995636, 0.10911434, ..., 0.00628325, 0.        ,
        0.        ]], dtype=float32)

In [ ]:
X_da, labels_da, knn_data_da = utils.fine_tuned_TL('fusar', data_augmentation=True, num_epochs=2)

In [10]:
X_zs, labels_zs, knn_data_zs = utils.zero_shot_TL('fusar', data_augmentation=False)

Using cache found in /Users/jameschapman/.cache/torch/hub/pytorch_vision_v0.10.0
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X0_5_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X0_5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
knn_data_zs[1][:, -1]

array([0.3475616 , 0.2934168 , 0.52518052, ..., 0.31965578, 0.45735806,
       0.63850933])